In [2]:
import spacy
from nltk import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import pandas as pd

In [3]:
with open ("archive (2)\dialogs.txt", "r") as file:
    text = file.read()

data = pd.read_csv("archive (2)\dialogs.txt", sep="\t", names = ["question", "answer"])

In [196]:
load_model = spacy.load("en_core_web_sm")

def preproc(text):
    doc = load_model(text)
    return " ".join([token.lemma_ for token in doc])

data["q_prep"] = data.agg({"question": preproc})

C:\Users\1\AppData\Local\Temp\ipykernel_15276\1624131598.py:7: FutureWarning: using <function preproc at 0x00000246D7687F60> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  data["q_prep"] = data.agg({"question": preproc})


In [117]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random

In [8]:
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(data["q_prep"])

<3725x1806 sparse matrix of type '<class 'numpy.float64'>'
	with 21812 stored elements in Compressed Sparse Row format>

In [ ]:
vect_questions = vectorizer.transform(data["q_prep"]).toarray()
data["q_vect"] = pd.Series(vect_questions[i] for i in range(len(vect_questions)))

In [191]:
def chat_bot(text:str):
    user_message = vectorizer.transform([text])
    user_message = user_message.toarray()

    cos_sim_list = cosine_similarity(user_message, vect_questions)
    loc_info = np.where(cos_sim_list == max(cos_sim_list[0]))

    answer_list = data["answer"].iloc[loc_info[1]]
    if len(answer_list) > 1:
        answer_ind = random.choice(range(len(answer_list)))
        return answer_list.iloc[answer_ind]

    return answer_list.iloc[0]

In [227]:
chat_bot("hello")

"okay. i'll return your pen when i'm done."

In [228]:
chat_bot("how are you?")

'so how do you plan to become rich?'

In [202]:
chat_bot("do you need a pen?")

'no. i already have a pen.'

In [208]:
chat_bot("go do your homework")

'but, mom!'

In [210]:
chat_bot("do you like football?")

'oh, yes, i really like it.'

In [211]:
chat_bot("i love you")

'i love you, too.'

In [213]:
chat_bot("i hate you")

'so do i.'

In [214]:
chat_bot("do you hate yourself?")

"of course. it's not a hard job."

In [215]:
chat_bot("don't hate yourself")

'i started school recently.'

In [216]:
chat_bot("do you hate school?")

'i go to pcc.'

In [222]:
chat_bot("pcc?")

'do you like it there?'

In [225]:
chat_bot("i don't know what it is")

'maybe i know, but how could you know?'